In [1]:
!pip install accelerate

  Using cached accelerate-0.27.2-py3-none-any.whl.metadata (18 kB)
Using cached accelerate-0.27.2-py3-none-any.whl (279 kB)


In [2]:
import torch
from diffusers import AnimateDiffPipeline, LCMScheduler, MotionAdapter
from diffusers.utils import export_to_gif
import warnings


e:\Hecktobyte\story_video_creation\myenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
e:\Hecktobyte\story_video_creation\myenv\Lib\site-packages\diffusers\utils\outputs.py:63: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


In [3]:
adapter = MotionAdapter.from_pretrained("wangfuyun/AnimateLCM", torch_dtype=torch.float16)
pipe = AnimateDiffPipeline.from_pretrained("emilianJR/epiCRealism", motion_adapter=adapter, torch_dtype=torch.float16)
pipe.scheduler = LCMScheduler.from_config(pipe.scheduler.config, beta_schedule="linear")

pipe.load_lora_weights("wangfuyun/AnimateLCM", weight_name="AnimateLCM_sd15_t2v_lora.safetensors", adapter_name="lcm-lora")
pipe.set_adapters(["lcm-lora"], [0.8])

pipe.enable_vae_slicing()
pipe.enable_model_cpu_offload()

Fetching 15 files:   0%|          | 0/15 [00:00<?, ?it/s]e:\Hecktobyte\story_video_creation\myenv\Lib\site-packages\huggingface_hub\file_download.py:1006: UserWarning: Not enough free disk space to download the file. The expected file size is: 3438.17 MB. The target location C:\Users\Rahim\.cache\huggingface\hub only has 3389.67 MB free disk space.
  warnings.warn(
e:\Hecktobyte\story_video_creation\myenv\Lib\site-packages\huggingface_hub\file_download.py:1006: UserWarning: Not enough free disk space to download the file. The expected file size is: 3438.17 MB. The target location C:\Users\Rahim\.cache\huggingface\hub\models--emilianJR--epiCRealism\blobs only has 3389.67 MB free disk space.
  warnings.warn(


In [ ]:
output = pipe(
    prompt="make walking in jungle ",
    negative_prompt="bad quality, worse quality, low resolution",
    num_frames=16,
    guidance_scale=2.0,
    num_inference_steps=6,
    generator=torch.Generator("cpu").manual_seed(0),
)

frames = output.frames[0]
export_to_gif(frames, "animatelcm.gif")